In [ ]:
import torch

In [2]:
X_train = torch.tensor([
    [-1.2, 3.1], 
    [-0.9, 2.9], 
    [-0.5, 2.6], 
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8], 
    [2.6, -1.6]
])

y_test = torch.tensor([0, 1])

In [3]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, index):
        x_item = self.X[index]
        y_item = self.y[index]

        return x_item, y_item

    def __len__(self):
        return self.y.shape[0] 

In [5]:
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

len(train_ds), len(test_ds)

(5, 2)

In [21]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_ds, 
    batch_size=2, 
    shuffle=True, 
    num_workers=0, 
    drop_last=True
)

test_loader = DataLoader(
    test_ds, 
    batch_size=2, 
    shuffle=True, 
    num_workers=0, 
    drop_last=True
)

for index, (x, y) in enumerate(train_dataloader):
    print(f"{index} - {x},{y}")

0 - tensor([[ 2.3000, -1.1000],
        [-0.5000,  2.6000]]),tensor([1, 0])
1 - tensor([[-0.9000,  2.9000],
        [ 2.7000, -1.5000]]),tensor([0, 1])


In [8]:
# Now let's compare performance of a neural network on a CPU vs GPU

class NeuralNetworkCPU(torch.nn.Module):
    def __init__(self, num_inputs: int, num_outputs: int) -> None:
        super().__init__()
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs

        self.layers = torch.nn.Sequential(
            # 1st Hidden Layer
            torch.nn.Linear(self.num_inputs, 50, device="cpu"),
            torch.nn.ReLU(),

            # 2nd Hidden Layer
            torch.nn.Linear(50, 20), 
            torch.nn.ReLU(),

            # Output Layer
            torch.nn.Linear(20, self.num_outputs)
        )

    def number_of_parameters(self) -> int:
        """
        This function returns the number of trainable parameters in the model
        """

        model = NeuralNetworkCPU(self.num_inputs, self.num_outputs)

        num_params = sum([p.numel() for p in model.parameters() if p.requires_grad == True])

        return num_params
    
    def forward(self, x) -> list:
        """
        This function enables the forward pass of the neural networks
        """

        logits = self.layers(x)

        return logits

In [9]:
import torch.nn.functional as F

torch.manual_seed(123)

model = NeuralNetworkCPU(num_inputs=2, num_outputs=2)

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()

    for batch_idx, (features, labels) in enumerate(train_dataloader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        print(f"""Epoch: {epoch+1:03d}/{num_epochs:03d} | Batch {batch_idx:03d}/{len(train_dataloader):03d} | Train loss: {loss:.2f}""")

model.eval()

Epoch: 001/003 | Batch 000/002 | Train loss: 0.82
Epoch: 001/003 | Batch 001/002 | Train loss: 0.27
Epoch: 002/003 | Batch 000/002 | Train loss: 0.25
Epoch: 002/003 | Batch 001/002 | Train loss: 0.06
Epoch: 003/003 | Batch 000/002 | Train loss: 0.02
Epoch: 003/003 | Batch 001/002 | Train loss: 0.02


NeuralNetworkCPU(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)

In [10]:
model.number_of_parameters()

1212

In [14]:
# Making predictions
with torch.no_grad():
    output = model(X_train)
    print(output)

probabilities = torch.softmax(output, dim=1)
print(probabilities)

predicitons = torch.argmax(probabilities, dim=1)
predicitons

tensor([[ 4.1435, -3.3049],
        [ 3.6839, -2.9598],
        [ 3.0230, -2.4603],
        [-1.9138,  1.8427],
        [-2.2214,  2.1564]])
tensor([[9.9942e-01, 5.8205e-04],
        [9.9870e-01, 1.3004e-03],
        [9.9586e-01, 4.1388e-03],
        [2.2831e-02, 9.7717e-01],
        [1.2397e-02, 9.8760e-01]])


tensor([0, 0, 0, 1, 1])

In [15]:
sum(predicitons == y_train)

tensor(5)

In [23]:
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total_examples = 0.0

    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)

        probabilities = torch.softmax(logits, dim=1)

        predicitons = torch.argmax(probabilities, dim=1) # Setting dim 0 will return max of each row

        compare = labels == predicitons

        correct += sum(compare)

        total_examples += len(compare)

    return (correct / total_examples).item()

In [24]:
compute_accuracy(model, test_loader)

1.0

In [26]:
# Saving and loading the saved model

torch.save(model.state_dict(), "model.pth")

In [27]:
model = NeuralNetworkCPU(2, 2)

model.load_state_dict(torch.load("model.pth"))

C:\Users\athar\AppData\Local\Temp\ipykernel_18640\2666273379.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>